In [1]:
### Piezometer installed in the ditch ###
# monitor the ditch water level
# piezometer installed before the second weir
# AP689

In [2]:
import pandas as pd
import xlrd
import os
import numpy as np
import glob
import matplotlib.pyplot as plt
#from pandasql import sqldf
import datetime
import csv
import pdb
from tqdm.notebook import tqdm
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import FuncFormatter
def minor_formatter(x, ind):
    # only format if we don't overlap with a major tick
    if np.mod(x, 1) < .1:
        return ''
    return '{:.1f}'.format(np.mod(x, 1))
def alternate_formatter(x, ind):
    rm = np.mod(x, 1)
    if np.abs(rm) < .1:
        return '{:d}'.format(int(x))
    return '{:.1f}'.format(rm)
%matplotlib ipympl

In [3]:
%matplotlib ipympl
path1 = os.getcwd()
path = os.path.join(path1, 'data', 'raw_data')
#climate data from  www.dacom.nl
file = os.path.join(path, 'dacom_weather_data.csv')
clim=pd.read_csv(file,sep=';')
clim['date'] = pd.to_datetime(clim['date'], format = '%m/%d/%Y')
print(clim)
climday = clim.groupby(clim.date.dt.floor('d')).agg({'precipitation': ['sum']})
climday.columns  = ['_'.join(col) for col in climday.columns.values]
climday.reset_index(inplace=True)
print(climday)

           date      time  air_temperature  rel_humidity  precipitation  \
0    2022-05-17  12:00 AM            15.57         90.16            0.0   
1    2022-05-17   1:00 AM            15.20         95.71            0.0   
2    2022-05-17   2:00 AM            15.11         97.59            0.0   
3    2022-05-17   3:00 AM            14.91         92.53            0.0   
4    2022-05-17   4:00 AM            14.72        100.00            0.0   
...         ...       ...              ...           ...            ...   
4970 2022-12-10   1:00 AM            -2.46        100.00            0.0   
4971 2022-12-10   2:00 AM            -2.53        100.00            0.0   
4972 2022-12-10   3:00 AM            -2.36        100.00            0.0   
4973 2022-12-10   4:00 AM            -2.13        100.00            0.0   
4974 2022-12-10   5:00 AM            -1.73        100.00            0.0   

      wind_speed  wind_direction  radiation  radiation_short  
0           0.54          164.34    

In [4]:
path1 = os.getcwd()
datapath =  os.path.join(path1, 'Data_Ditch')
xls_files = glob.glob(datapath + "\*.xlsx") # change to csv 

baro_ind='AP825'
ind = 0
diver_names={}
alldf={}
for i,f in tqdm(enumerate(xls_files),total=len(xls_files)):
    # read the csv file
    df = pd.read_excel(f)
    #print(df)
    # remove double copied parts of the logged data from the raw data files
    df=df.drop_duplicates()  
    df=df.assign(Date=df.TimeStamp.dt.round('H'))
    
    df['Date']=pd.to_datetime(df['Date'],dayfirst=True)
    df=df.loc[(df['TimeStamp'] > '2022-10-18 14:27:20')]
    dfdaily=df.resample('D', on = 'Date').mean()
    
    if baro_ind in f:
        baro_data=df
        baro_data=baro_data.loc[(baro_data['TimeStamp'] > '2022-08-02 12:00:00')]
        #baro_data=baro_data.append.df# this tests for substrings
    if ind == 0:
        #First data frame loaded
        GWdata = df
        GWdata_daily = dfdaily

print(GWdata)
print(baro_data)


     MonitoringPoint SerialNumber           TimeStamp  Pressure  Temperature  \
1974            ILVO        AP825 2022-10-18 15:00:00  1047.675    28.373333   
1975            ILVO        AP825 2022-10-18 16:00:00  1047.575    26.560000   
1976            ILVO        AP825 2022-10-18 17:00:00  1047.400    22.106667   
1977            ILVO        AP825 2022-10-18 18:00:00  1047.875    13.616667   
1978            ILVO        AP825 2022-10-18 19:00:00  1048.425    11.966667   
...              ...          ...                 ...       ...          ...   
3213            ILVO        AP825 2022-12-09 06:00:00  1026.950    -1.900000   
3214            ILVO        AP825 2022-12-09 07:00:00  1026.750    -2.853333   
3215            ILVO        AP825 2022-12-09 08:00:00  1026.825    -3.563333   
3216            ILVO        AP825 2022-12-09 09:00:00  1027.375    -3.276667   
3217            ILVO        AP825 2022-12-09 10:00:00  1028.000    -1.416667   

      Conductivity  WaterLevel  WaterL

In [5]:
#Soil elevation: 19.21 mTAW
#Well height above the ground: 230cm
#Rope length: 251cm

In [7]:
wellinfo = { 'SerialNumber' : ['AP689'],
            'soil_mTAW' : [19.21],
            'well_height' : [2.30],
            'top_well_cm' :[2151],
            'RL' : [251],}

alldata={}
date1=datetime.date(2022,8,1)
date2=datetime.date(2022,12,10)
ftsz=16
fig, ax= plt.subplots(figsize=(10,5))
axb=ax.twinx()


ddd=GWdata.reset_index()
print(ddd)
dataall=baro_data.merge(ddd[['Date','Pressure']], left_on = 'TimeStamp', right_on='Date')
dataall[dataall['Pressure_y'].notna()]
dataall = dataall[dataall['Pressure_x']<dataall['Pressure_y']]
dataall["WL"]=(dataall['Pressure_y']-dataall['Pressure_x'])
dataall=dataall.reset_index()
dataall["RL"]=251
dataall["top_well_cm"]=2151
dataall["GWL"]=-dataall['RL']+dataall['top_well_cm']+dataall["WL"]    
ax.plot(dataall.Date_x,(dataall["GWL"]/100), color='#ff7f0e', label="Ditch water level [mTAW]")
ax.set_ylim([18.75,20.75])
ax.set_ylabel('GWL[mTAW]', fontsize= ftsz-2)
ax.hlines(y=19.21, xmin = date1, xmax = date2, color='#026b2e', linestyle = '--', label="Soil elevation [mTAW]")
ax.set_title('Evolution of the ditch water level', fontsize=ftsz)
ax.tick_params(axis='x', which='both', labelsize=ftsz-2,rotation=60)
ax.tick_params(axis='y', which='both', labelsize=ftsz-2)
ax.legend(loc='lower left', fontsize=8)
alldata=dataall

axb.bar(climday['date'],climday.precipitation_sum)
axb.set_ylim([0, 30])
axb.invert_yaxis()
axb.set_ylabel('Precipitation [mm]', fontsize= ftsz-2)
axb.tick_params(axis='x', which='both', labelsize=ftsz-2)
axb.tick_params(axis='y', which='both', labelsize=ftsz-2)

axb.set_xlim([date1,date2])

print(dataall)

fig.savefig('DitchWaterLevel.png')   # save the figure to file


FigureCanvasNbAgg()

      index MonitoringPoint SerialNumber           TimeStamp  Pressure  \
0      1974            ILVO        AP825 2022-10-18 15:00:00  1047.675   
1      1975            ILVO        AP825 2022-10-18 16:00:00  1047.575   
2      1976            ILVO        AP825 2022-10-18 17:00:00  1047.400   
3      1977            ILVO        AP825 2022-10-18 18:00:00  1047.875   
4      1978            ILVO        AP825 2022-10-18 19:00:00  1048.425   
...     ...             ...          ...                 ...       ...   
1239   3213            ILVO        AP825 2022-12-09 06:00:00  1026.950   
1240   3214            ILVO        AP825 2022-12-09 07:00:00  1026.750   
1241   3215            ILVO        AP825 2022-12-09 08:00:00  1026.825   
1242   3216            ILVO        AP825 2022-12-09 09:00:00  1027.375   
1243   3217            ILVO        AP825 2022-12-09 10:00:00  1028.000   

      Temperature  Conductivity  WaterLevel  WaterLevelReference  \
0       28.373333           NaN         NaN